In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
import holodeck.gravwaves
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
resamp = holo.population.PM_Resample(0.2)
pop = holo.population.Pop_Illustris(mods=resamp)
print(pop.size)

hards = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(),
]

# Composite Hardening Model

In [ ]:
ecc = holo.population.PM_Eccentricity()
pop = holo.population.Pop_Illustris(mods=ecc)

hards = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(),
]

In [ ]:
evo = holo.evolution.Evolution(pop, hards, debug=True)
evo.evolve()

In [ ]:
rads = np.logspace(-4, 4, 100) * PC
num_hard = len(hards)
params = ['eccen', 'dadt', 'dedt',] + [f'_dadt_{ii}' for ii in range(num_hard)] + [f'_dedt_{ii}' for ii in range(num_hard)]
# lin_interp_list = evo._LIN_INTERP_PARS + params
lin_interp_list = params
data = evo.at('sepa', rads, params=params, lin_interp=lin_interp_list)

In [ ]:
fig, axes = plot.figax(
    figsize=[10, 6], nrows=2,
    xlabel=r"Separation $[\mathrm{pc}]$", ylabel=r'Timescale $[\mathrm{yr}]$',
)


frac = utils.frac_str(evo.scafa[:, -1] < 1.0, )
xx = rads / PC

ax = axes[0]
ax.set(title=f"coalescing = {frac}", ylim=[1e2, 1e12])
dadt = data['dadt']
vals = np.fabs(dadt)
vals = rads / vals
vals = vals / YR
confs = utils.quantiles(vals, sigmas=[-1, 0, 1], axis=0)
confs = confs.T
med, *confs = confs[[1, 0, 2]]

ax.plot(xx, med, 'k-')
ax.fill_between(xx, *confs, color='k', alpha=0.2)

ax = axes[1]
ax.set_yscale('symlog', linthresh=1e3)
ax.set(ylim=[-1e12, 1e12])
vals = data["dedt"]
vals = data['eccen'] / vals
vals = vals / YR
confs = utils.quantiles(vals, sigmas=[-1, 0, 1], axis=0)
confs = confs.T
med, *confs = confs[[1, 0, 2]]
ax.plot(xx, med, 'k-', alpha=0.25)
ax.fill_between(xx, *confs, color='k', alpha=0.1)

vals = data['eccen']
xx = rads / PC
confs = utils.quantiles(vals, sigmas=[-1, 0, 1], axis=0)
confs = confs.T
med, *confs = confs[[1, 0, 2]]

tw = ax.twinx()
tw.set(yscale='linear', label='eccentricity')
col = 'r'
tw.plot(xx, med, ls='--', color=col, alpha=0.2)
tw.plot(xx, confs[0], ls=':', color=col, alpha=0.1)
tw.plot(xx, confs[-1], ls=':', color=col, alpha=0.1)
tw.fill_between(xx, *confs, color=col, alpha=0.05)

for ii in range(num_hard):
    try:
        lab = hards[ii].__name__
    except AttributeError:
        lab = hards[ii].__class__.__name__
        
    vals = np.fabs(data[f"_dadt_{ii}"])
    vals = rads / vals
    vals = vals / YR

    confs = utils.quantiles(vals, sigmas=[-1, 1], axis=0).T
    axes[0].fill_between(xx, *confs, alpha=0.2, label=lab)

    # eccentricity
    if evo.eccen is None:
        continue

    vals = data[f"_dedt_{ii}"]
    vals = data['eccen'] / vals
    vals = vals / YR

    confs = utils.quantiles(vals, sigmas=[-1, 1], axis=0).T
    axes[1].fill_between(xx, *confs, alpha=0.2)

axes[0].legend(loc='lower right')
plt.show()

# Simplest Custom Population and Hardening Model

In [ ]:
SIZE = 100

class Pop(holo.population._Population_Discrete):

    def _init(self):
        self.mass = (10.0 ** np.random.uniform(6, 10, (SIZE, 2))) * MSOL
        self.sepa = (10.0 ** np.random.uniform(1, 3, SIZE)) * 1e3 * PC
        self.scafa = np.random.uniform(0.25, 0.75, SIZE)
        return

class Hard(holo.evolution._Hardening):

    def dadt_dedt(self, evo, step, *args, **kwargs):
        dadt = -(PC/YR) * np.ones(evo.size)
        dedt = None
        return dadt, dedt

# Construct instances
pop = Pop()
hard = Hard()
evo = holo.evolution.Evolution(pop, hard)
# evolve population
evo.evolve()

Plot the resulting evolution of each binary showing coalescences

In [ ]:
fig, ax = plot.figax()

idx = ()
xx = evo.tage[idx]/GYR
xx = xx - xx[:, 0, np.newaxis]
yy = evo.sepa[idx]/PC

ax.plot(xx.T, yy.T)

plt.show()